TODO: 
- get better data, this is not realistic (predictions are too wild)

# Intro

Goal: create a function that draws lift charts. 

In [4]:
import numpy as np
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt

import tweedie # for tweedie random sample

# Chart Summary

lift chart shows two lines: one for predicted and one for actual

first, the data is sorted by the predicted values

then, we sum up the actual and predicted values in a number of bins

cutoff points should be by policy level (an alternative approach would be to keep the sum of insured years constant across the groups)

the predicted line will be monotonic, and the actual will be a jagged line along it

# Sample Data Generation

Create 100 groups with different tweedie distributions, and make the predictions kinda align with the group averages, but with a large enough random adjustment to make it imperfect. 

In [38]:
df = pd.DataFrame.from_dict({'feature' : np.repeat(range(1,101),100)})
df['feature'] = df['feature'] * 1000

In [39]:
tweedie.tweedie(mu=10, p=1.5, phi=20).rvs(1)[0]

0.0

In [40]:
# note: this takes a bit of time even with only 10k rows
df['actu'] = df.apply(lambda row : tweedie.tweedie(mu=row['feature'], p=1.5, phi=20).rvs(1)[0], axis = 1)

In [43]:
averages = df.groupby('feature').agg({'actu': np.mean})
averages

,actu
feature,
1000,1023.858586
2000,1992.885585
3000,3050.272427
4000,3929.129012
5000,5057.273470
...,...
96000,97158.874094
97000,95024.913990
98000,98969.703627


In [73]:
df['pred_error'] = np.random.normal(loc = 0, scale = 2000, size = 10000)

In [74]:
df['pred'] = df['feature'] + df['pred_error']

In [75]:
df['pred'] = df['pred'].clip(100,None)

# Functions

First, we need a function that converts the data into a lift chart table.

In [76]:
input_df = df.copy()

In [77]:
input_df.sort_values('pred', inplace = True)

In [ ]:
number_of_bins = 15

In [81]:
input_df['rank'] = range(1,len(input_df['pred'])+1)

In [ ]:
input_df['']

In [82]:
input_df

,feature,actu,pred_error,pred,rank
0,1000,1023.624935,-1120.102032,100.000000,1
93,1000,542.400908,-1158.711480,100.000000,2
89,1000,717.396669,-3434.656762,100.000000,3
88,1000,0.000000,-2001.593506,100.000000,4
87,1000,67.632094,-1694.769381,100.000000,5
...,...,...,...,...,...
9912,100000,122516.144185,3429.872431,103429.872431,9996
9938,100000,77520.432484,3569.111428,103569.111428,9997
9926,100000,78856.186105,3853.043002,103853.043002,9998
9970,100000,46016.090236,4216.360666,104216.360666,9999
